In [1]:
from bs4 import BeautifulSoup as BS
import re
from time import sleep
import requests
from yandexfreetranslate import YandexFreeTranslate
from multiprocessing.dummy import Pool as ThreadPool
from tqdm.notebook import tqdm

In [2]:
# name = 'ultragene-warlord'
name = 'lord-of-mysteries-2-circle-of-inevitability'
# name = 'the-primal-hunter'

In [3]:
url = 'https://novelbjn.novelupdates.net/ajax/chapter-option'

params = {
    'novelId': name,
}

res_links = requests.get(url=url, params=params)
page = BS(res_links.text, 'html.parser')
links = list(map(lambda x: x.get('value'), page.find('body').find_all('option')))


In [4]:
book = []

In [5]:
start = 0
for index, url in tqdm(list(enumerate(links))[start:]):
    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36',
    }

    res_page = requests.get(
        url,
        headers=headers,
    )
    page = BS(res_page.text, 'html.parser').find('div', id='chr-content')
    

    
    paragraphs = [tag.text.strip() for tag in page.find('div').next_siblings if tag.name in ['p', 'h4', 'h3']]
    book.append(paragraphs)

  0%|          | 0/920 [00:00<?, ?it/s]

SSLError: [SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:2559)

In [ ]:
def _worker(pool_list: tuple[str, tqdm]) -> str:
    queue, pbar = pool_list
    if not queue.strip():
        pbar.update(1)
        return ''
    yt = YandexFreeTranslate(api = "ios")
    
    for _ in range(10):
        try:
            text = yt.translate("en", "ru", queue)
        except Exception:
            sleep(0.5)
            continue
        else:
            break
    else:
        text = queue
        pbar.write(str(queue))
    pbar.update(1)
    return text
    

def run_translate(lst: list[str], pbar: tqdm)-> list[str]:
    pbar.reset(len(lst))
    pool_list = list(zip(lst, [pbar]*len(lst)))
    with ThreadPool(8) as pool:
        workreturn = pool.map(_worker, pool_list)
    return workreturn

In [ ]:
new_book = []

In [ ]:
start = 0
pbar1 = tqdm(desc='page')
pbar2 = tqdm(desc='row')
pbar1.reset(len(book[start:]))
for index, page in list(enumerate(book))[start:]:
    try:
        new_page = run_translate(page, pbar2)
        new_book.append(new_page)
    except Exception:
        print(index)
        break
    pbar1.update()
pbar1.close()
pbar2.close()
    

page: 0it [00:00, ?it/s]

row: 0it [00:00, ?it/s]

In [ ]:
from ebooklib import epub
from random import randint

# Функция для преобразования страницы в HTML
def page_to_html(page, title):
    content_lines = page[:]
    # Формирование HTML-кода
    html = f"<h1>{title}</h1>"
    for line in content_lines:
        html += f"<p>{line}</p>"
    return html


title = name

# Создание новой книги
epub_book = epub.EpubBook()

# Установка метаданных
epub_book.set_identifier(f"id{randint(100000,1000000)}")
epub_book.set_title(title)
epub_book.set_language("ru")

# Автор
epub_book.add_author("None")
# epub_book.spine.append('nav')
# Обработка каждой страницы
for i, page in enumerate(new_book):
    # Создание главы
    chapter_title = f'Глава {i+1:04d}'
    chapter_content = page_to_html(page, chapter_title)
    
    chapter = epub.EpubHtml(title=chapter_title, file_name=f'chap_{i+1:04d}.xhtml', lang='ru')
    chapter.content = chapter_content
    
    epub_book.add_item(chapter)
    
    # Добавление в оглавление
    epub_book.toc.append(chapter)
    # Упаковка всех глав в книгу
    epub_book.spine.append(chapter)


# Добавление оглавления
epub_book.add_item(epub.EpubNcx())
epub_book.add_item(epub.EpubNav())

# Сохранение книги
epub.write_epub(f'{name}.epub', epub_book)


In [ ]:
# from ebooklib import epub

# book = epub.EpubBook()

# # set metadata
# book.set_identifier("id123456")
# book.set_title("Sample book")
# book.set_language("en")

# book.add_author("Author Authorowski")
# book.add_author(
#     "Danko Bananko",
#     file_as="Gospodin Danko Bananko",
#     role="ill",
#     uid="coauthor",
# )

# # create chapter
# c1 = epub.EpubHtml(title="Intro", file_name="chap_01.xhtml", lang="hr")
# c1.content = (
#     "<h1>Intro heading</h1>"
#     "<p>Zaba je skocila u baru.</p>"
#     '<p><img alt="[ebook logo]" src="static/ebooklib.gif"/><br/></p>'
# )

# # create image from the local image
# # image_content = open("ebooklib.gif", "rb").read()
# # img = epub.EpubImage(
# #     uid="image_1",
# #     file_name="static/ebooklib.gif",
# #     media_type="image/gif",
# #     content=image_content,
# # )

# # add chapter
# book.add_item(c1)
# # add image
# # book.add_item(img)

# # define Table Of Contents
# book.toc = (
#     epub.Link("chap_01.xhtml", "Introduction", "intro"),
#     (epub.Section("Simple book"), (c1,)),
# )

# # add default NCX and Nav file
# book.add_item(epub.EpubNcx())
# book.add_item(epub.EpubNav())

# # define CSS style
# style = "BODY {color: white;}"
# nav_css = epub.EpubItem(
#     uid="style_nav",
#     file_name="style/nav.css",
#     media_type="text/css",
#     content=style,
# )

# # add CSS file
# book.add_item(nav_css)

# # basic spine
# book.spine = ["nav", c1]

# # write to the file
# epub.write_epub("test.epub", book, {})